
Import Dependedncies



In [46]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

Data colleaction and analysis

In [47]:
# load csv file to create dataframe
diabetes_data = pd.read_csv('/content/diabetes.csv')

In [48]:
#print first 5 rows of dataframe
diabetes_data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [49]:
# Give the shape of the dataframe
diabetes_data.shape

(100000, 9)

In [50]:
# Get standard mesurements
diabetes_data.describe()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,41.885856,0.07485,0.039420,27.320767,5.527507,138.058060,0.085000
std,22.516840,0.26315,0.194593,6.636783,1.070672,40.708136,0.278883
min,0.080000,0.00000,0.000000,10.010000,3.500000,80.000000,0.000000
25%,24.000000,0.00000,0.000000,23.630000,4.800000,100.000000,0.000000
50%,43.000000,0.00000,0.000000,27.320000,5.800000,140.000000,0.000000
75%,60.000000,0.00000,0.000000,29.580000,6.200000,159.000000,0.000000
max,80.000000,1.00000,1.000000,95.690000,9.000000,300.000000,1.000000


In [51]:
#Value counts
diabetes_data['diabetes'].value_counts()

,count
diabetes,
0,91500
1,8500


From this dataset, 8500 people are diabetic and 91500 people are not

In [52]:
diabetes_data['gender'].value_counts()

,count
gender,
Female,58552
Male,41430
Other,18


There are 58552 women and 41430 men

In [53]:
group_by_gender = diabetes_data.groupby('gender')

females = group_by_gender.get_group('Female')
males = group_by_gender.get_group('Male')

# Diabetic scores on Females
print('Females: ',females['diabetes'].value_counts())

# Diabetic sscores on Males
print('Males: ', males['diabetes'].value_counts())

Females:  diabetes
0    54091
1     4461
Name: count, dtype: int64
Males:  diabetes
0    37391
1     4039
Name: count, dtype: int64


4461 out of 58552 women are diabetic

4093 out of 41430 men are diabetic

Drop rows where Smoking History == 'No Info'

In [54]:
diabetes_data = diabetes_data[diabetes_data['smoking_history'] != 'No Info']

Drop rows where Gender == 'Other'

In [55]:
diabetes_data = diabetes_data[diabetes_data['gender'] != 'Other']

Display Head

In [56]:
diabetes_data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
5,Female,20.0,0,0,never,27.32,6.6,85,0


Encode Non numeric data
---

Gender
---

Female: 0

Male: 1




Smoking History
---

Never smoked: 0

Current smoker: 1

Former smoker: 2

Has ever smoked before: 3

Not currently smoking: 4

In [57]:
#Gender
diabetes_data['gender'] = diabetes_data['gender'].replace({'Female': 0, 'Male': 1})

#Smoking History
diabetes_data['smoking_history'] = diabetes_data['smoking_history'].replace({'never': 0, 'current': 1, 'former': 2, 'ever': 3, 'not current': 4})

print(diabetes_data)

       gender   age  hypertension  heart_disease  smoking_history    bmi  \
0           0  80.0             0              1                0  25.19   
2           1  28.0             0              0                0  27.32   
3           0  36.0             0              0                1  23.45   
4           1  76.0             1              1                1  20.14   
5           0  20.0             0              0                0  27.32   
...       ...   ...           ...            ...              ...    ...   
99992       0  26.0             0              0                0  34.34   
99993       0  40.0             0              0                0  40.69   
99997       1  66.0             0              0                2  27.83   
99998       0  24.0             0              0                0  35.42   
99999       0  57.0             0              0                1  22.43   

       HbA1c_level  blood_glucose_level  diabetes  
0              6.6                 

<ipython-input-57-7e209078d7c6>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  diabetes_data['gender'] = diabetes_data['gender'].replace({'Female': 0, 'Male': 1})
<ipython-input-57-7e209078d7c6>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  diabetes_data['smoking_history'] = diabetes_data['smoking_history'].replace({'never': 0, 'current': 1, 'former': 2, 'ever': 3, 'not current': 4})


Get Mean value of each column after grouping by diabetes column

In [58]:
diabetes_data.groupby('diabetes').mean()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
diabetes,,,,,,,,
0,0.384781,44.757243,0.077845,0.034433,1.005689,27.956466,5.395414,132.970294
1,0.473886,61.060034,0.260573,0.149305,1.178541,32.216953,6.933366,193.622197


The average diabetic in this data is 60 years of age with a higher HbA1c level, glucose level and hypertension level

In [59]:
A = diabetes_data.drop(columns=['diabetes'], axis=1)
B = diabetes_data['diabetes']

print(A)
print(B)

       gender   age  hypertension  heart_disease  smoking_history    bmi  \
0           0  80.0             0              1                0  25.19   
2           1  28.0             0              0                0  27.32   
3           0  36.0             0              0                1  23.45   
4           1  76.0             1              1                1  20.14   
5           0  20.0             0              0                0  27.32   
...       ...   ...           ...            ...              ...    ...   
99992       0  26.0             0              0                0  34.34   
99993       0  40.0             0              0                0  40.69   
99997       1  66.0             0              0                2  27.83   
99998       0  24.0             0              0                0  35.42   
99999       0  57.0             0              0                1  22.43   

       HbA1c_level  blood_glucose_level  
0              6.6                  140  
2  

Data Preprocessing

In [60]:
#Data standardization
scaler = StandardScaler()

#Fit calculates the mean and standard deviation and transform uses that data to
# help keep values between 0 and 1
A = scaler.fit_transform(A)

print(A)

[[-0.8072818   1.71205298 -0.32944696 ... -0.49636265  0.94540938
   0.0087797 ]
 [ 1.23872482 -0.9492171  -0.32944696 ... -0.16947121  0.1238867
   0.43566023]
 [-0.8072818  -0.53979094 -0.32944696 ... -0.76340073 -0.51507538
   0.36451348]
 ...
 [ 1.23872482  0.99555719 -0.32944696 ... -0.09120143  0.1238867
   0.36451348]
 [-0.8072818  -1.15393019 -0.32944696 ...  1.07363707 -1.42787835
  -0.93984369]
 [-0.8072818   0.53495275 -0.32944696 ... -0.91994029  0.94540938
  -1.17699954]]


Train Test Splitting

In [61]:
a_train, a_test, b_train, b_test = train_test_split(A, B, test_size=0.2, stratify=B, random_state=2)
print(A.shape, a_train.shape, a_test.shape)

(64172, 8) (51337, 8) (12835, 8)


Train the model

In [62]:
#Instantate the classifier
model = svm.SVC(kernel='linear')

#Train the model
model.fit(a_train, b_train)

SVC(kernel='linear')

Model Evaluation: Testing model accuracy

In [63]:
#Check accuracy of training data
a_train_prediction = model.predict(a_train)
training_data_accuracy = accuracy_score(a_train_prediction, b_train)

print('Accuracy score of the training data: ', training_data_accuracy)

Accuracy score of the training data:  0.9498412451058691


In [64]:
#Check accuracy score of the testing data
a_test_prediction = model.predict(a_test)
testing_data_accuracy = accuracy_score(a_test_prediction, b_test)

print('Accuracy score of the testing data: ', testing_data_accuracy)

Accuracy score of the testing data:  0.9504479937670433


Predictive system using trained model

In [65]:
#This patient is diabetic. This cell checks the model's accuracy
input = (1,69.0,0,1,0,29.34,8.2,155)

#As numpy array
input_np = np.asarray(input)

#Reshape array to get prediction for just one instance
input_np_reshaped = input_np.reshape(1, -1)

#Standardize input data
std_data = scaler.transform(input_np_reshaped)
print(std_data)

prediction = model.predict(std_data)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[[ 1.23872482  1.149092   -0.32944696  4.50067162 -0.75446471  0.14053851
   2.40589414  0.36451348]]
[1]
The person is diabetic


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Packaging the model with pickle

In [66]:
import pickle

with open('/content/diabetes_prediction_model.pkl', 'wb') as f:
  pickle.dump(model, f)